In [1]:
from itertools import combinations
from tqdm.auto import tqdm
import numpy as np
import sample as sampler
import utils
import attacker

In [2]:
model_path = "/mnt/e/BaiduNet/ms1mv3_arcface_r100_fp16"
data_insight_path = "/mnt/e/BaiduNet/ms1m-retinaface-t1/ms1m-retinaface-t1"
data_fei_retinaface_path = "/mnt/e/Downloads/FEI/originalimages_retinaface_122"
data_fatial_path = data_fei_retinaface_path

In [3]:
import tempfile
# tempdir = '/tmp/last name of the model without suffix'
tempdir = '/tmp/' + model_path.split('/')[-1].split('.')[0]
tempdir

'/tmp/ms1mv3_arcface_r100_fp16'

In [4]:
from torchvision import transforms
from dataset.fatial_dataset import FatialDataset

fatial_dataset = FatialDataset(data_fatial_path, transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    lambda x: x * 2 - 1]), target_transform=lambda label: str(label))

In [5]:
from dataset.fatial_dataset_embedding_dict import FatialDataEmbeddingsDict
dataset_dict = FatialDataEmbeddingsDict(file_folder=tempdir, data_set=fatial_dataset, file_raw_name="labels_set_sphere.pickle")
dataset_dict.dump_embeddings(None, 10, None, True)

## Generate puzzle with each identity

In [6]:
identity_list = list(dataset_dict.keys())

In [7]:
# codeword parameters
dimension = 512
alpha = 16
n = 511
error_rate = 0.4    # angle(w + noise_1, w + noise_2) 
                    # approximate 2*arcsin(error_rate * \sqrt(2) / \sqrt(1 + error_rate^2) / 2)
print("Estimated angle between two noisy templates: {}".format(2 * np.arcsin(error_rate * np.sqrt(2) / np.sqrt(1 + error_rate**2) / 2) * 180 / np.pi))

Estimated angle between two noisy templates: 30.450314021355645


In [8]:
# generate puzzle
import ironmask
# random select a identity from identity_list
# use 0, 2, 5 as the query unprotected templates
cs = []
isometric_matrixes = []
for each_identity in tqdm(identity_list):
    each_isometric_matrixes = []
    each_cs = []
    for index in [0, 2, 5]:
        tmpcs = ironmask.sample_codeword(dimension, alpha)
        each_isometric_matrixes.append(ironmask.generate_secure_sketch(dataset_dict[each_identity][index], tmpcs))
        each_cs.append(tmpcs)
    cs.append(cs)
    isometric_matrixes.append(each_isometric_matrixes)

  0%|          | 0/200 [00:00<?, ?it/s]

In [23]:
# get the mean angle between each pair of templates, i.e. angle(w + noise_1, w + noise_2)

angles = []
for each_identity in tqdm(identity_list):
    each_identity_vector = dataset_dict[each_identity]
    ws = []
    for index in [0, 2, 5]:
        ws.append(each_identity_vector[index])
    for i, j in combinations(range(len(ws)), 2):
        angles.append(utils.get_angle_of_two_vectors(ws[i], ws[j]))
    

print("Test Mean Degree: {:.2f}{}".format(np.mean(angles), u'\u00b0'))

  0%|          | 0/200 [00:00<?, ?it/s]

Test Mean Degree: 28.56°


## solving original template if sampled matrix is correct

ensure the matrix generated by linear equation sampler is "correct" as in Definition 4.1 in paper.

svd solver

In [ ]:
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, cs, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="SVD", disable_tqdm=False, k_each_matrix=1)

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

lsa solver

In [28]:
runtimes_vec = []
success_times = 0
for each_identity in (pbar:=tqdm(identity_list)):
    each_isometric_matrixes = []
    each_cs = []
    for index in [0, 2, 5]:
        tmpcs = ironmask.sample_codeword(dimension, alpha)
        each_isometric_matrixes.append(ironmask.generate_secure_sketch(dataset_dict[each_identity][index], tmpcs))
        each_cs.append(tmpcs)
    assume_vector, b, run_times = attacker.solve_puzzle_with_n_matrix_known_places(each_isometric_matrixes, each_cs, dimension, alpha,  threshold=49, max_rtimes=2000, algorithm="LSA", disable_tqdm=False, k_each_matrix=170, error_rate = error_rate * 3.0, return_runtimes = True)
    if b is not None and (np.allclose(b, each_cs[0]) or np.allclose(b, -each_cs[0])):
        success_times += 1
    pbar.set_postfix({"Success Rate": success_times / (pbar.n + 1)})
    runtimes_vec.append(run_times)

  0%|          | 0/200 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

...

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

Solving(LSA):   0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
# 保存结果和success times
import pickle
with open("runtimes_vec.pickle", "wb") as f:
    pickle.dump(runtimes_vec, f)
with open("success_times.pickle", "wb") as f:
    pickle.dump(success_times, f)

In [ ]:
# 加载结果
import pickle
with open("runtimes_vec.pickle", "rb") as f:
    runtimes_vec = pickle.load(f)
with open("success_times.pickle", "rb") as f:
    success_times = pickle.load(f)

print("Success Rate: {:.2f}%".format(success_times / len(identity_list) * 100))
print("Average Run Times: {:.2f}".format(np.mean(runtimes_vec)))

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

The result is:  True


## solving original template without preposition that sampled matrix is "correct"

In [ ]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000,
                                                        algorithm="SVD", disable_tqdm=False, k_each_matrix=1, scale=100.0)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

In [ ]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, scale=1000.0, error_rate=error_rate * 3.0)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))